In [5]:
import cv2
import numpy as np
import os

In [7]:
class SceneChangeDetector:
    def __init__(self, threshold=30, min_scene_len=40):
        """
        Initialize the scene change detector.

        Args:
            threshold: Sensitivity threshold for detecting scene changes.
            min_scene_len: Minimum length of a scene in frames.
        """
        self.threshold = threshold
        self.min_scene_len = min_scene_len
        
    def find_scene_changes(self, video_path):
        """
        Detect scene changes in the given video.

        Args:
            video_path: Path to the video file.

        Returns:
            List of scene changes with their corresponding frame indices and timecodes.
        """
        cap = cv2.VideoCapture(video_path)
        fps = cap.get(cv2.CAP_PROP_FPS)
        frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        scene_changes = []

        prev_frame = None
        frame_idx = 0
        last_scene_frame = 0

        while True:
            ret, frame = cap.read()
            if not ret:
                break

            frame_idx += 1
            # Convert to grayscale for processing.
            gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            if prev_frame is not None:
                # Calculate absolute difference between frames.
                diff = cv2.absdiff(prev_frame, gray_frame)
                diff_mean = np.mean(diff)

                # Check if the difference exceeds the threshold.
                if diff_mean > self.threshold and (frame_idx - last_scene_frame) > self.min_scene_len:
                    # Save the scene change.
                    timecode = self.frame_to_timecode(frame_idx, fps)
                    scene_changes.append((frame_idx, timecode))
                    last_scene_frame = frame_idx

            prev_frame = gray_frame

        cap.release()
        return scene_changes
    
    @staticmethod
    def frame_to_timecode(frame_idx, fps):
        """
        Convert frame index to timecode format.

        Args:
            frame_idx: Index of the frame.
            fps: Frames per second of the video.

        Returns:
            Timecode in the format HH:MM:SS.
        """
        total_seconds = frame_idx / fps
        hours = int(total_seconds // 3600)
        minutes = int((total_seconds % 3600) // 60)
        seconds = int(total_seconds % 60)
        return f"{hours:02}:{minutes:02}:{seconds:02}"


In [8]:
# Example usage:
if __name__ == "__main__":
    video_path = os.environ['PROJECT_DIRECTORY'] + 'Dataset/MSR-VTT/TrainVal/video0.mp4' # Replace with your video file path
    detector = SceneChangeDetector(threshold=30, min_scene_len=40)

    scenes = detector.find_scene_changes(video_path)
    if not scenes:
        print("No scenes detected.")
    else:
        print("Detected scenes:")
        for scene_idx, timecode in scenes:
            print(f"Scene change at frame {scene_idx}, timecode {timecode}")

Detected scenes:
Scene change at frame 53, timecode 00:00:02
Scene change at frame 176, timecode 00:00:07
Scene change at frame 287, timecode 00:00:11
